## LSTM

### Exploding Gradients
Exploding Gradients의 해결방법으로 **Gradients Clipping**이 존재한다.  
Gradients Clipping에대한 식은 아래와 같다.  
<p>$$if ||\hat{g}|| \ge threshold$$</p>
<p>$$\hat{g} =  \frac{threshold}{||\hat{g}||}\hat{g}$$</p>

**Parameter 설명**  
$$\hat{g}$$
사용하는 모든 매개변수의 기울기를 하나로 모은 것
$$||\hat{g}||$$
<span>$$\hat{g}$$ </span>에 L2 norm 적용
$$threshold$$
문턱값

위와같이 문턱값을 초과하면 문턱값으로 값을 대입하는 것으로 **Exploding Gradients**를 해결할 수 있는 것을 알 수 있다.  

In [1]:
import numpy as np


dW1 = np.random.rand(3, 3) * 10
dW2 = np.random.rand(3, 3) * 10
grads = [dW1, dW2]
max_norm = 5.0


def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate


print('before:', dW1.flatten())
clip_grads(grads, max_norm)
print('after:', dW1.flatten())

before: [3.77033332 3.51282312 9.93686421 2.50423331 5.45508533 1.91662074
 6.38966985 5.06224907 3.17974005]
after: [0.88867921 0.82798326 2.34214958 0.59025552 1.28578046 0.45175343
 1.50606492 1.19318774 0.74947455]


- σ(시그모이드): 0 ~ 1의 범위를 가지게 출력형태를 바꿔주며 데이터를 얼마만큼 통과시킬지를 정하는 비율
- tanh(하이퍼 볼릭 탄젠트): -1 ~ 1의 범위를 가지게 출력형태를 바꿔주며 실질적인 정보의 비율
- Output gate: <span>$$o = \sigma (W_{xh_o}x_t +W_{hh_o}h_{t-1} + b_{h_o})$$</span>: 다음 시간의 Hidden Layer에서 얼만큼 중요한가를 나타내는 상수
- Forget gate: <span>$$f_t = \sigma (W_{xh_f}x_t +W_{hh_f}h_{t-1} + b_{h_f})$$</span>: 과거 정보를 잊기 위한 게이트(0 ~ 1사이의 값을 가지는 Scalar로서 얼만큼 잊을지 비율로서 표현)
-  <span>$$g$$</span>: <span>$$tanh(W_{xh_g}x_t +W_{hh_g}h_{t-1} + b_{h_g})$$</span>: tanh를 사용하여 현재 LSTM Layer에서의 실질적인 정보의 비율
- Input gate: <span>$$i_t = \sigma (W_{xh_i}x_t +W_{hh_i}h_{t-1} + b_{h_i})$$</span>: 현재 정보를 기억하기 위한 게이트(0 ~ 1사이의 값을 가지는 Scalar로서 얼만큼 기억할 비율로서 표현)
- <span>$$c_t$$</span>: 기억 셀로서 과거로부터 시각 t까지에 필요한 모든 정보가 저장된 Cell, <span>$$c_t = f \odot c_{t-1} + g \odot i $$</span>
- <span>$$h_t$$</span>: <span>$$o \odot tanh(c_t)$$</span>: Hidden Layer의 출력 o가 Sigmoid의 Output으로서 상수이므로 <span>$$\odot$$ </span>사용

위와 같은 LSTM의 망을 **Affine 변환**을 통해 빠르게 계산수행을 위해 아래와 같이 망을 최종적으로 구성한다.  
<div><img src="http://i.imgur.com/73zzDsC.png" height="100%" width="100%" /></div>

In [2]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''

        Parameters
        ----------
        Wx: 입력 x에 대한 가중치 매개변수(4개분의 가중치가 담겨 있음)
        Wh: 은닉 상태 h에 대한 가장추 매개변수(4개분의 가중치가 담겨 있음)
        b: 편향（4개분의 편향이 담겨 있음）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev):
        Wx, Wh, b = self.params
        N, H = h_prev.shape

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b

        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f)
        g = np.tanh(g)
        i = sigmoid(i)
        o = sigmoid(o)

        c_next = f * c_prev + g * i
        h_next = o * np.tanh(c_next)

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next)
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next)

        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i)
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2)

        dA = np.hstack((df, dg, di, do))

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

## Time LSTM
T개분의 시계열 데이터를 한꺼번에 처리하는 계층

In [3]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            hs[:, t, :] = self.h

            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0

        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        return dxs

    def set_state(self, h, c=None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None

위에서 정의한 **Time LSTM**을 활용하여 Softmax직전까지의 Rnnlm을 구성하는 코드이다.  
즉, Embedding -> Time LSTM -> Time Affine까지의 계층 구현이다.  

In [4]:
import sys
sys.path.append('..')
from common.time_layers import *
from common.base_model import BaseModel


class Rnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=100, hidden_size=100):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.lstm_layer.reset_state()

이제 구현한 Rnnlm을 Train하는 Code이다.  
이전 Post에서의 Simple Rnnlm과 다른점은 **Exploding Gradients을 해결하기 위하여 Exploding Clipping을 사용한 것 이다.**  


In [6]:
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
from dataset import ptb


# 하이퍼파라미터 설정
batch_size = 20
wordvec_size = 100
hidden_size = 100  # RNN의 은닉 상태 벡터의 원소 수
time_size = 35     # RNN을 펼치는 크기
lr = 20.0
max_epoch = 4
max_grad = 0.25

# 학습 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_test, _, _ = ptb.load_data('test')
vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

# 모델 생성
model = Rnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

# 기울기 클리핑을 적용하여 학습
trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad,
            eval_interval=20)
trainer.plot(ylim=(0, 500))

# 테스트 데이터로 평가
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('테스트 퍼플렉서티: ', ppl_test)

# 매개변수 저장
model.save_params()

Done
Done
| 에폭 1 |  반복 1 / 1327 | 시간 0[s] | 퍼플렉서티 9999.68


KeyboardInterrupt: 

현재 Time LSTM에서 여러가지 기법을 사용하여 좀 더 Model의 퍼블렉서티의 값이 낮아지게 하는 것을 목표로 한다.  

**1. Layer Depth 증가**  
Layer Depth가 증가하게 되면 비선형성이 더욱 증가되어 Data를 예측하는 성능이 높아지게 된다.  
하지만 이러한 Layer Depth를 증가시키면 크게 2가지의 문제가 발생하게 된다.  
1. Layer Depth가 깊어질수록 Hyper Parameter의 수가 증가하게 되며, 학습에 사용할 데이터 양이 제한적일 경우 Overfitting에 빠질 위험이 있다.
2. Layer Depth가 증가하게 되면 연산량이 늘어나게 된다.

현재 사용하는 PTB데이터셋의 언어 모델에서는 LSTM의 층 수는 2~4정도일때 좋은 결과를 만든다고 하여 2층으로 선정하였다.  

**2. Dropout**  
DropOut Dropout은 Overfitting을 막기위한 방법으로 뉴럴 네트워크가 학습중일때, 랜덤하게 뉴런을 꺼서 학습함으로써, 학습이 학습용 데이터로 치우치는 현상을 막아준다.  
<a href="https://wjddyd66.github.io/dl/2019/08/31/NeuralNetwork-(5)-Others.html">Dropout의 자세한 내용</a><br>

**LSTM 의 경우 Dropout의 위치가 중요하게 된다. Dropout의 위치에 따라서 timestep의 정보가 지워져서 시계열 데이터를 사용하는데 어려움을 겪을 수 있기 때문이다.**  

따라서 LSTM에서 권장하는 Dropout의 위치는 다음과 같다.  
<div><img src="https://nmhkahn.github.io/assets/RNN-Reg/p1-dropout.png" height="250" width="600" /></div>

위 그림에서 점선은 dropout이 적용된 연결이고 실선은 dropout이 적용이 되지 않은 것이다. 제시한 방법을 곱씹어보면 이전 timestep에서 온 정보는 dropout 하지 말고, **현재 timestep에서 들어온 입력값 혹은 이전 레이어의 값만 dropout** 하는 의미이다.  
현재 timestep에서 이전 레이어인 <span>$$h_t^{l-1}$$</span>에 dropout을 적용했는데 과거 timestep의 정보를 지우지 않게 하기 위해 recurrent한 연결 (과거 timestep)에 dropout을 적용하지 않은 것으로 보인다. 다시 정리하면, 일반적인 dropout을 적용한다면 먼 과거의 정보를 잃어버려 학습하는데 어려움을 갖지만, **non-recurrent한 연결만 dropout을 적용하면 과거 중요 정보를 희생하지 않아도 regularization을 사용**할 수 있다.

위의 내용을 정리하면 즉, LSTM의 결과 <span>$$c_t^{l}, h_t^{l}$$</span>에서 이전 timestep의 내용을 가지고 있는 <span>$$c_t^{l}$$</span>에 dropout을 적용하게 되면 timestep의 정보가 지워질 수 있다는 것 이다.  

**3. 가중치 공유(Weight Tying)**  
**Weight Tying** 란 입력-임베딩 레이어와 출력 과 softmax 레이어 사이의 가중치 매트릭스 공유 즉, 두 개의 가중치 행렬을 사용하는 대신 하나의 가중치 행렬 만 사용하는 방법이다.  
즉 Embedding을 Encoding이라고 생각하게 되면, Encoding과 Decoding의 방식을 같게한다고 생각할 수 있다.  
실제로 가중치 공유를 하게되면 다음과 같은 2가지 장점을 생각할 수 있다.
1. Hyper Parameter의 수가 감소하게 되여 Overfitting에 빠질 위험 감소
2. Update해야할 Weight감소로 인하여 학습 속도 개선



**실제 구현**  
- LSTM 계층의 다층화(2층)
- Dropout 사용
- 가중치 공유

In [7]:
from common.time_layers import *
from common.np import *  # import numpy as np
from common.base_model import BaseModel


class BetterRnnlm(BaseModel):
    def __init__(self, vocab_size=10000, wordvec_size=650,
                 hidden_size=650, dropout_ratio=0.5):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx1 = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh1 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b1 = np.zeros(4 * H).astype('f')
        lstm_Wx2 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_Wh2 = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b2 = np.zeros(4 * H).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.layers = [
            TimeEmbedding(embed_W),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx1, lstm_Wh1, lstm_b1, stateful=True),
            TimeDropout(dropout_ratio),
            TimeLSTM(lstm_Wx2, lstm_Wh2, lstm_b2, stateful=True),
            TimeDropout(dropout_ratio),
            TimeAffine(embed_W.T, affine_b)  # weight tying!!
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layers = [self.layers[2], self.layers[4]]
        self.drop_layers = [self.layers[1], self.layers[3], self.layers[5]]

        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, xs, train_flg=False):
        for layer in self.drop_layers:
            layer.train_flg = train_flg

        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts, train_flg=True):
        score = self.predict(xs, train_flg)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()

**결과 확인**  
이전 Train과 달리 검증 데이터로 퍼플렉서티를 평가하고, 그 값이 나빠졌을 경우에만 학습률을 낮추는 방법으로 Train을 하였다.  

In [8]:
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ==============================================
# config.GPU = True
# ==============================================
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 학습 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size,
                time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('검증 퍼플렉서티: ', ppl)

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr

    model.reset_state()
    print('-' * 50)


# 테스트 데이터로 평가
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('테스트 퍼플렉서티: ', ppl_test)

Done
| 에폭 1 |  반복 1 / 1327 | 시간 3[s] | 퍼플렉서티 9999.90


KeyboardInterrupt: 